<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/huggingface/hf_nlp_03_fine_tune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 微调模型



### 处理数据

下面是在 PyTorch 中的一批上训练文本分类器

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
#  SentencePiece是一个用于文本分词的库，它可以将文本分割成子词（subword）单元
# !pip install transformers

In [ ]:
import torch
# 从transformers库中导入AdamW优化器、AutoTokenizer自动分词器和AutoModelForSequenceClassification自动文本分类模型。
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
# 使用指定的检查点初始化一个自动分词器，这个分词器将用于将文本序列转换为模型能理解的格式。
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# 使用指定的检查点初始化一个自动文本分类模型，这个模型将用于分类任务。
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
# 使用分词器处理文本序列列表，
# padding=True：进行填充（padding）以确保序列长度一致
# truncation=True：截断（truncation）长的序列以适应模型的最大长度限制
# return_tensors="pt"：返回PyTorch张量
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
# 使用了PyTorch创建了一个包含两个1的张量作为标签
batch["labels"] = torch.tensor([1, 1])

# 创建一个AdamW优化器，它将用于在训练过程中更新模型的参数。
optimizer = AdamW(model.parameters())

# 将批次数据传递给模型，并获取模型输出的损失值。使用了**batch来将字典解包为关键字参数。
loss = model(**batch).loss
# 对损失值进行反向传播，计算梯度。
loss.backward()

# 使用优化器根据计算得到的梯度更新模型的参数
optimizer.step()

上面是在两个句子上训练模型，不会产生很好的结果。为了获得更好的结果，需要准备更大的数据集。

#### 从hub加载数据

In [5]:
from datasets import load_dataset

# 使用load_dataset函数来加载名为"glue"的基准测试中的"mrpc"数据集。
# 这里的"glue"指的是GLUE基准测试，它是一个广泛用于评估自然语言处理模型性能的集合，包含了多个不同的文本分类任务。
# "mrpc"是GLUE基准测试中的一个任务，即MRPC数据集，它包含句子对和它们的复述标签。
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

 加载数据集，得到DatasetDict 对象，这是一个特殊类型的字典，其中包含了训练集、验证集和测试集（如果可用）。通常，数据集会被分为至少两个部分：'train'和'validation'（或'val'），有时还包括'test'集。每一个都包含几列（ sentence1 、 sentence2 、 label 和 idx ）和num_rows，它们是每个集合中的元素数量。

 加载数据集后，你可以使用datasets库提供的方法来进一步处理和分析数据，例如对数据进行清洗、分词、编码等，以便输入到模型中进行训练和评估。

 下面试着方位训练数据集里面的内容：

In [11]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

查看每个字段的类型：`raw_train_dataset.features`。可以看出label标签的含义：

0 对应于 not_equivalent ， 1 对应于 equivalent 。

In [6]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [11]:
raw_train_dataset[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

In [10]:
raw_validation_dataset = raw_datasets["validation"]

raw_validation_dataset[87]



{'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
 'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
 'label': 0,
 'idx': 812}

#### 预处理数据

In [6]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# 错误示范
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

我们不能向上面这样分别传递模型两个文本，并推理两个句子是否意思相似。我们需要将两个序列作为一对处理，并应用适当的预处理。幸运的是，BERT模型支持这种方式。

In [14]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [101, 2023, 2003, 1996, 2353, 2028, 102]}

In [12]:
inputs = tokenizer(raw_train_dataset[15]["sentence1"], raw_train_dataset[15]["sentence2"])
inputs

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

 - input_ids ：需要传递给模型的输入
 - attention_mask: 注意力是否需要忽略
 - token_type_ids: 告诉模型输入的哪个部分是第一个句子，哪个部分是第二个句子。

In [8]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

我们已经看到了我们的分词器如何处理一对句子，我们可以用它来分词我们的整个数据集：
我们可以使用一次将数据集加载到内存的方式。（仅当您有足够的 RAM 来在标记化期间存储整个数据集时，它才会起作用）

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)





**（推荐方式）使用 Dataset.map() 方法，将数据保持为数据集格式：**

In [17]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

tokenized_datasets 将是一个新的数据集，其中包含了原始数据集的文本经过分词处理后的结果，以及任何其他在 tokenize_function 中定义的预处理步骤的结果。这个新的数据集可以用于模型的训练和评估。

#### 动态填充

负责将样本放在批处理中的函数称为`collate function`。Transformers 库通过 DataCollatorWithPadding 为我们提供了`collate function`，它将正确数量的填充应用于我们想要批处理在一起的数据集的项目。

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

下面我们来测试一下，DataCollatorWithPadding动态填充能力：

In [22]:
# 从tokenized_datasets数据集的训练集中选取前8个样本。
samples = tokenized_datasets["train"][:8]

# 使用字典推导式来创建一个新的字典samples，它包含原始样本中除去"idx"、"sentence1"和"sentence2"键的所有项。
# 这意味着新的samples字典将只包含经过分词器处理后的键值对，例如"input_ids"、"token_type_ids"和"attention_mask"等
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

# 遍历samples字典中的"input_ids"列表，并为列表中的每个元素计算长度。
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

我们得到了不同长度的样本，从 32 到 67。动态填充意味着该批次中的样本应全部填充到长度 67，这是批次内的最大长度。如果没有动态填充，所有样本都必须填充到整个数据集中的最大长度，或者模型可以接受的最大长度。

In [23]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

#### 加载其他数据

在 GLUE SST-2 (是GLUE基准测试中的一个任务，专注于文本的情感分析)数据集演练一遍数据处理，它有点不同，因为它是由单个句子而不是成对的句子组成。

> 加载数据第二个参数可以用的值: ['ax', 'cola', 'mnli', 'mnli_matched', 'mnli_mismatched', 'mrpc', 'qnli', 'qqp', 'rte', 'sst2', 'stsb', 'wnli']

In [27]:
# 加载数据
raw_datasets = load_dataset("glue","sst2")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [28]:
# 查看训练数据
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [30]:
# 查看训练数据字段类型，label标签：0-负面情感，1-正面情感
raw_train_dataset.features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [35]:
# 数据预处理

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_ssl2_function(example):
    return tokenizer(example["sentence"], truncation=True,padding=True)

tokenized_datasets = raw_datasets.map(tokenize_ssl2_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [40]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# samples = tokenized_datasets["train"][:1]

# batch = data_collator(tokenized_datasets["train"][0])


### 使用 Trainer API 微调模型

Transformers 提供了一个 Trainer 类来帮助您微调它在数据集上提供的任何预训练模型。

下面是“处理数据”一节中的概要：

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

#### Training(微调训练)

定义 Trainer 之前：
- 第一步是定义一个 TrainingArguments 类，该类将包含 Trainer 将用于训练和评估的所有超参数。入参是保存训练模型的目录以及沿途的检查点，其他保持默认。

- 第二步是定义我们的模型

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
!pip install -U transformers[torch]
# !pip show accelerate
# !pip uninstall  accelerate


**注意**:在执行`TrainingArguments(output_dir="test-trainer")`包如下错误:
> "Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`"

并且执行了如下命令,accelerate版本也大于0.20.0
```
 !pip install transformers[torch]
 !pip install accelerate -U
!pip show accelerate
```
升级transformers[torch]版本到最新得以解决:
> !pip install -U transformers[torch]


In [2]:
from transformers import TrainingArguments
# 训练参数
training_args = TrainingArguments(output_dir="test-trainer")

TrainingArguments是transformers库中的一个类，用于存储和传递训练过程中的各种配置参数。这个类提供了丰富的选项，以控制模型训练的各个方面，包括但不限于学习率、批次大小、训练轮数、评估策略等。
以下是TrainingArguments类的常用参数：

- output_dir: 训练过程中生成的输出（如模型权重、配置文件等）的存储路径。
- evaluation_strategy: 评估策略，可以是基于epoch的评估或基于steps的评估。
- learning_rate: 初始学习率。
- per_device_train_batch_size: 每个设备上的训练批次大小。
- per_device_eval_batch_size: 每个设备上的评估批次大小。
- gradient_accumulation_steps: 梯度累积的步数，用于控制实际的批次大小。
- num_train_epochs: 训练的总轮数。
- max_steps: 如果设置了num_train_epochs，则训练的最大步数，这可以用于提前终止训练。
- weight_decay: 权重衰减的强度，用于正则化以防止过拟合。
- adam_beta1: AdamW优化器的β1参数。
- adam_beta2: AdamW优化器的β2参数。
- adam_epsilon: AdamW优化器的ε参数。
- max_grad_norm: 梯度裁剪的最大范数。
- lr_scheduler_type: 学习率调度器的类型，如线性、余弦退火等。
- warmup_steps: 学习率预热的步数，通常用于预热阶段的线性增长。
- total_num_update_steps: 总的更新步数，用于某些学习率调度器。
- logging_dir: 存储日志的目录。
- logging_steps: 日志记录的步数间隔。
- save_strategy: 保存模型的策略，可以是每epoch或每一定步数。
- load_best_model_at_end: 训练结束时是否加载最佳模型。
- metric_for_best_model: 用于评估“最佳模型”的指标。
- greater_is_better: 指标是否越大越好。
- disable_tqdm: 是否禁用进度条。
- seed: 随机种子，用于确保结果的可复现性。
- local_rank: 当使用分布式训练时的本地进程排名。
- fp16: 是否使用混合精度训练。
- fp16_opt_level: 混合精度优化的级别。
- gradient_checkpointing: 是否使用梯度检查点技术来减少内存使用。
- local_rank: 用于多GPU训练的本地进程ID。
- overwrite_output_dir: 是否覆盖输出目录中的现有内容。
- dataloader_num_workers: 数据加载时使用的进程数。
- dataloader_pin_memory: 如果设置为True，数据加载器将使用锁页内存。

In [5]:
from transformers import AutoModelForSequenceClassification
# 定义模型
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import Trainer
# 创建训练器
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

> **注意**，当您像我们在这里所做的那样传递 tokenizer 时， Trainer 使用的默认 data_collator 将是前面定义的 DataCollatorWithPadding

In [7]:
# 微调模型
trainer.train()

Step,Training Loss
500,0.502500
1000,0.255000


TrainOutput(global_step=1377, training_loss=0.3062528085258445, metrics={'train_runtime': 222.4039, 'train_samples_per_second': 49.478, 'train_steps_per_second': 6.191, 'total_flos': 405114969714960.0, 'train_loss': 0.3062528085258445, 'epoch': 3.0})

#### Evaluation（评估）

上面的微调不会告诉您模型的表现有好坏，是因为：
- 没有为 Trainer 提供 compute_metrics() 函数来在所述评估期间计算指标
- 没有告诉 Trainer 在训练期间通过将 evaluation_strategy 设置为 "steps" （评估每个 eval_steps ）或 "epoch" （在每个时期结束时进行评估）。

In [14]:
import numpy as np
import evaluate

# compute_metrics：用于计算模型在评估数据集上的性能指标
def compute_metrics(eval_preds):
  # 加载评估指标:
    metric = evaluate.load("glue", "mrpc")
    # 分离模型输出和标签: eval_preds是一个包含两个元素的元组，
    # 其中第一个元素logits是模型的原始输出（即模型最后一层的线性变换输出），
    # 第二个元素labels是数据集中的真实标签。
    logits, labels = eval_preds
    # 计算预测类别: 使用np.argmax函数沿最后一个轴（axis=-1）计算logits中最大值的索引
    predictions = np.argmax(logits, axis=-1)
    # 计算评估指标: predictions参数是模型预测的类别，而references参数是对应的真实标签。
    return metric.compute(predictions=predictions, references=labels)

In [12]:
# evaluation_strategy 设置为 "epoch"
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.371126,0.857843,0.901024
2,0.527200,0.459970,0.845588,0.893401
3,0.279600,0.774146,0.848039,0.894915


TrainOutput(global_step=1377, training_loss=0.3341553964386319, metrics={'train_runtime': 231.7036, 'train_samples_per_second': 47.492, 'train_steps_per_second': 5.943, 'total_flos': 405114969714960.0, 'train_loss': 0.3341553964386319, 'epoch': 3.0})

### Trainer 类微调训练内部实现

了解如何在不使用 Trainer 类的情况下获得与上一节相同的结果。

In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

#### 训练准备

- 对 tokenized_datasets 应用一些后处理，以处理 Trainer 自动为我们完成的一些事情：
  - 删除与模型不需要的值对应的列（例如 sentence1 和 sentence2 列）
  - 将列 label 重命名为 labels （因为模型期望参数命名为 labels ）
  - 设置数据集的格式，以便它们返回 PyTorch 张量而不是列表。

- 数据处理完后，定义用来迭代批次的数据加载器

In [16]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [17]:
from torch.utils.data import DataLoader
# 加载数据
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [18]:
# 快速检查数据处理中是否有错误
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

In [19]:
from transformers import AutoModelForSequenceClassification
# 实例化一个模型
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.9363, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [21]:
from transformers import AdamW
# 优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler
# 学习率调度器
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### 循环训练

In [22]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

#### 评估每批次的循环

In [25]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8725490196078431, 'f1': 0.9109589041095889}

### 一个案例

使用BERT模型处理一个文本分类任务

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

# 加载数据集
dataset = load_dataset("glue", "sst2")

# 初始化tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 使用tokenizer对数据集中的文本进行编码
def encode(examples):
    return tokenizer(examples["sentence"], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(encode, batched=True)

# 加载预训练模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 初始化DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 定义训练参数（这里只是示例，实际参数需要根据任务和模型调整）
from transformers import TrainingArguments

import numpy as np
import evaluate

# compute_metrics：用于计算模型在评估数据集上的性能指标
def compute_metrics(eval_preds):
  # 加载评估指标:
    metric = evaluate.load("glue", "mrpc")
    # 分离模型输出和标签: eval_preds是一个包含两个元素的元组，
    # 其中第一个元素logits是模型的原始输出（即模型最后一层的线性变换输出），
    # 第二个元素labels是数据集中的真实标签。
    logits, labels = eval_preds
    # 计算预测类别: 使用np.argmax函数沿最后一个轴（axis=-1）计算logits中最大值的索引
    predictions = np.argmax(logits, axis=-1)
    # 计算评估指标: predictions参数是模型预测的类别，而references参数是对应的真实标签。
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 创建训练器
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    # 使用DataCollatorWithPadding来准备批次数据
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 训练模型
trainer.train()

# 在验证集上评估模型
trainer.evaluate()